<a href="https://colab.research.google.com/github/namwootree/Basic_Skill/blob/main/Web_Crawling/%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%8B%AC%ED%99%94_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### 한글 폰트 설치

In [ ]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [ ]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic Eco


In [ ]:
# 라이브러리 추가
import urllib.request # request 대체 가능
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

import json
import glob
import sys
import os
from datetime import datetime
from random import shuffle


### 0. API란? 

- API(application programming interface)     
  운영체제나 시스템, 애플리케이션, 라이브러리 등을 활용해 응용 프로그램을 작성할 수 있게 하는 다양한 인터페이스를 의미합니다. Window API, Java API, HTML5 API, Android API 등이 있습니다. (네이버 open api 가이드)

- 웹 API   
  웹 API는 웹 애플리케이션 개발에서 다른 서비스에 요청을 보내고 응답을 받기 위해 정의된 명세를 말합니다. 예를 들어 블로그 API를 이용하면 블로그에 접속하지 않고도 다른 방법으로 글을 올릴 수 있습니다. 그 외에 우체국의 우편번호 API, 구글과 네이버의 지도 API등 유용한 API들이 많아서, 요즘은 홈페이지 구축이나 추가개편 시 따로 추가로 개발하지 않고 이런 오픈 API를 가져와 사용하는 추세입니다. (위키백과)

- 오픈 API    
  API 중에서 플랫폼의 기능 또는 콘텐츠를 외부에서 웹 프로토콜(HTTP)로 호출해 사용할 수 있게 개방(open)한 API를 의미합니다. 네이버 개발자센터에서 제공하고 있는 지도, 검색을 비롯 기계 번역, 캡차, 단축 URL 등 대부분 API는 HTTP로 호출할 수 있는 오픈 API에 해당합니다. (네이버 open api 가이드)

- 그외 API 관련 용어
https://developers.naver.com/docs/common/openapiguide/apiterms.md#api%EC%9D%98-%EA%B8%B0%EB%B3%B8

### 1. 네이버 API

네이버 오픈API는 네이버 플랫폼의 기능을 외부 개발자가 쉽게 이용할 수 있게 웹 또는 SDK 형태로 공개한 기술들입니다. 네이버 오픈API로 활용할 수 있는 기술에는 네이버 아이디로 로그인과 지도, 검색이 있으며, Clova의 음성 인식 기술과 음성 합성 기술, 얼굴 인식 기술, Papago의 기계 번역 기술 등이 있습니다.

이 중에서 네이버 데이터랩 API와 Papago 기계번역 API를 활용해보겠습니다. 

##### 1-1. 네이버 데이터 랩 API

가장 먼저  Open API를 사용하기 위한 키를 발급받아야 합니다.   
https://developers.naver.com/apps/#/register?defaultScope=datalab    
위 링크에서 API 키를 발급받아 주세요

In [ ]:
naver_id = input('네이버 ID를 입력해주세요 : ') # 애플리케이션 등록시 발급 받은 값 입력
naver_key = input('네이버 API 키를 입력해주세요 : ') # 애플리케이션 등록시 발급 받은 값 입력

##### 네이버 데이터 랩 소개

- 네이버 데이터 랩은 궁금한 주제어를 설정하고, 하위 주제어에 해당하는 검색어를 콤마(,)로 구분입력하면,. 입력한 단어의 추이를 하나로 합산하여 해당 주제가 네이버에서 얼마나 검색되는지 조회할 수 있는 서비스입니다. 
https://datalab.naver.com/keyword/trendSearch.naver   

- 웹에서는 주제어 5개까지만 가능하지만 API를 활용하면 더 많은 주제들을 비교할 수 있습니다. 
- 네이버 데이터랩 API 개발가이드 : https://developers.naver.com/docs/datalab/search/

In [ ]:
# 개발가이드에 나온 샘플 
# 정해진 요청 형식에 맞게 요청을 해야 합니다 ! 
# 원하는 데이터를 body에 반영하여 요청하면 됩니다. 

import os
import sys
import urllib.request
client_id = naver_id
client_secret = naver_key
url = "https://openapi.naver.com/v1/datalab/search"
body = "{\"startDate\":\"2019-01-01\",\"endDate\":\"2020-03-15\",\"timeUnit\":\"month\",\"keywordGroups\":[{\"groupName\":\"한글\",\"keywords\":[\"한글\",\"korean\"]},{\"groupName\":\"영어\",\"keywords\":[\"영어\",\"english\"]}],\"device\":\"pc\",\"ages\":[\"1\",\"2\"],\"gender\":\"f\"}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

NameError: ignored

### 코스피 상장 회사 목록
- 2019년 1월 1일부터 현재까지 코스피 상장 회사들의 검색량을 비교해봅시다.

https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage

코드 참고: https://brunch.co.kr/@sukhyun9673/15#comment

In [ ]:
from urllib import request

#API 설정 : API키와 header 지정 
client_id = naver_id  
client_secret = naver_key

link = "https://openapi.naver.com/v1/datalab/search"
requested = request.Request(link)
requested.add_header("X-Naver-Client-Id",client_id)
requested.add_header("X-Naver-Client-Secret",client_secret)
requested.add_header("Content-Type","application/json")

NameError: ignored

In [ ]:
now = datetime.now().strftime("%Y-%m-%d") # 현재날짜 설정

# 개발가이드에 나와있는 요청 body와 같은 서식으로 딕셔너리 작성 
body_dict = {"startDate":"2019-01-01", 
               "endDate": now,
               "timeUnit":"month"}

NameError: ignored

In [ ]:
# 다운받은 상장 법인 목록의 회사명을 검색어로 설정 
df = pd.read_excel("/content/상장법인목록.xlsx")
names = df.회사명.values #df['회시명']


v_list = [{"groupName" : i, "keywords" : [i]} for i in names] # API 형식으로 만들기 

# 대표이름도 검색어로 설정하여 각 상장사의 검색량 얻기 
df["대표자명"] = df["대표자명"].apply(lambda x: re.sub(r'\(.*\)', '', x))
df["대표자명"]  = df["대표자명"].apply(lambda x: [re.compile('[^ㄱ-ㅣ가-힣]+').sub("",x)] if len(re.compile('[^ㄱ-ㅣ가-힣]+').sub("",x)) < 5 else re.findall(r"[\w']+", x)) 

for i in range(0, df.shape[0]):
    for j in df["대표자명"].values[i]:
        if "대표" not in j and j!= "":
            v_list[i]["keywords"].append(j) # 추가 키워드로 추가 

v_list

네이버 데이터랩의 검색어 트렌드는 요청된 기간 중 검색 횟수가 가장 높은 시점을 100으로 두고 나머지는 상대적 값으로 제공하고 있어서, 
랜덤으로 하나의 기준값을 정해놓고 다른 값들과 스케일 맞추는 작업 필요 

In [ ]:
####
# 네이버 데이터랩의 검색어 트렌드는 요청된 기간 중 검색 횟수가 가장 높은 시점을 100으로 두고 나머지는 상대적 값으로 제공하고 있어서, 
# 랜덤으로 하나의 기준값을 정해놓고 다른 값들과 스케일 맞추는 작업 필요  

shuffle(v_list) #랜덤화
standard = v_list[0] # 하나의 기준값 정하기 
standard_keyword = standard["groupName"]
list_use = v_list[1:] # 그외의 상장사들 


#### 
# 데이터 랩은 한번에 5개의 주제를 검색할 수 있으므로, 
# 상장사를 4개씩 분리하고, 각 리스트에 기준값을 추가 
split_list = [list_use[i:i+4] for i in range(0, len(list_use), 4)]

for i in split_list:
    i.append(standard)

split_list

In [ ]:
###스케일링할 회사의 Trend Index 뽑아내기
sample_body = body_dict
sample_body["keywordGroups"] = split_list[0]
sample_body = json.dumps(sample_body, ensure_ascii=False)
sample_response = request.urlopen(requested, data=sample_body.encode("utf-8"))

code = sample_response.getcode() #응답 코드 확인
if code == 200: #무사히 요청이 완료됨되면
    sample_response_body = sample_response.read()
    sample_scraped = sample_response_body.decode("utf-8")
else: 
    print ("Error Code:", code)

sample_result = json.loads(sample_scraped)

for i in sample_result["results"] :
    if i["title"] == standard_keyword:
        sample_standard = i["data"]

In [ ]:
import numpy as np
from tqdm  import tqdm
#'2019-01-01' 의 값을 Standard로 사용
scale = sample_standard[0]["ratio"]

df = {}

# df 딕셔너리에 standard값의 날짜와 ratio 추가  
df[standard_keyword] = np.array([i["ratio"] for i in sample_standard])
length = len(df[standard_keyword])
date = np.array([i["period"] for i in sample_standard])

# 5개씩 분리해놓은 split 리스트로 API 호출
for i in tqdm(split_list):
    body_dict["keywordGroups"] = i
    body = json.dumps(body_dict, ensure_ascii=False)
  
    response = request.urlopen(requested, data=body.encode("utf-8"))
    
    code = response.getcode() #응답 코드 확인
    if code == 200: #무사히 요청이 완료됨되면
        response_body = response.read()
        scraped = response_body.decode("utf-8")
    else: 
        print ("Error Code:", code)
    
    result = json.loads(scraped) # 각 split리스트의 값
    
    for i in result["results"]: # API 결과 
        if i["title"] == standard_keyword: # API결과 중 standard 값은 따로 저장 
            compare = i["data"]
    compare = compare[0]["ratio"]
    
    scaling = scale/compare   # 저장해놓은 standard값 / 해당 split 리스트에서의 standard값
    
    for i in result["results"]:
        if i["title"]!=standard_keyword:
            value = [j["ratio"]*scaling for j in i["data"]] # 위의 스케일링값으로 통일해서 df 딕셔너리에 결과 저장 
            if len(value)!=length:
                value+=np.abs(length-len(value)) * [value[-1]]
            df[i["title"]] = np.array(value)

df = pd.DataFrame(df)
df["date"] = date
df = df.set_index("date")

In [ ]:
df

In [ ]:
!pip install chart_studio
# pandas에서 바로 plotly로 그래프를 제작해주는 라이브러리 
!pip install cufflinks

In [ ]:
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px # seaborn보다 세련된 스타일
%matplotlib inline

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

import plotly.io as pio
pio.renderers.default = 'colab'

In [ ]:
import plotly.graph_objects as go
def plot_monthly_trend(df, company):
  """
  월 별 검색어 트렌드 그래프 출력
  """
  fig = px.line(df, x = df.index, y = df[company], title='월 별 검색어 트렌드', height = 500  , width = 1000, labels=dict(x='date'))
  fig.show()

In [ ]:
plot_monthly_trend(df, '삼성전자')

In [ ]:
c_list = list(df.sort_values(by = '2020-03-01', axis=1, ascending=False).columns[:10])  # 2020년 3월 1일 가장 검색이 많았던 회사 10개 추출 

In [ ]:
import plotly.graph_objects as go
def company_plot_monthly_trend(df, company_list):
  """
  월 별 검색어 트렌드 그래프 출력
  """
  filter = c_list
  df[filter].iplot(title = '월 별 검색어 트렌드 비교')

In [ ]:
company_plot_monthly_trend(df, c_list)

### 1-2. 파파고 API 활용

파파고 API를 활용하여 간단한 번역 프로그램을 만들어 보겠습니다.     
위에서 발급받은 clinet ID와 client secret 값을 활용하겠습니다 .  

- 공식 개발 가이드 : https://developers.naver.com/docs/nmt/reference/

번역 대상 언어 코드.  
  1.ko : 한국어  
  2.en : 영어  
  3.zh-CN : 중국어 간체  
  4.zh-TW : 중국어 번체  
  5.es : 스페인어  
  6.fr : 프랑스어  
  7.vi : 베트남어  
  8.th : 태국어  
  9.id : 인도네시아어  

ko<->en, ko<->zh-CN, ko<->zh-TW, ko<->es, ko<->fr, ko<->vi, ko<->th, ko<->id, en<->ja, en<->fr 조합만 가능

In [ ]:
import os
import sys
import urllib.request
client_id = naver_id # 개발자센터에서 발급받은 Client ID 값
client_secret = naver_key # 개발자센터에서 발급받은 Client Secret 값
encText = urllib.parse.quote("반갑습니다") # 번역할 언어 
data = "source=ko&target=en&text=" + encText  # 한 -> 영 번역 
url = "https://openapi.naver.com/v1/papago/n2mt"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

다른 언어로 번역한다면? 
한국어 -> 태국어

In [ ]:
import os
import sys
import urllib.request
client_id = naver_id # 개발자센터에서 발급받은 Client ID 값
client_secret = naver_key # 개발자센터에서 발급받은 Client Secret 값
encText = urllib.parse.quote("반갑습니다") # 번역할 언어 
data = "source=ko&target=th&text=" + encText   # target을 태국어로 변경 
url = "https://openapi.naver.com/v1/papago/n2mt"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()
if (rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

API를 활용하여 한국어 -> 영어 번역기를 만들어 봅시다

In [ ]:
import os
import sys
import urllib.request
from pprint import pprint

client_id = naver_id # 개발자센터에서 발급받은 Client ID 값
client_secret = naver_key # 개발자센터에서 발급받은 Client Secret 값

def get_translate():
    text = input('번역하고 싶은 문장을 입력해주세요 :')

    encText = urllib.parse.quote(text) 
    data = "source=ko&target=en&text=" + encText   
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if (rescode==200):
        response_body = response.read()
        print(json.loads(response_body)['message']['result']['translatedText'])
    else:
        print("Error Code:" , rescode)


In [ ]:
get_translate()